In [2]:
import pandas as pd
import numpy as np
import os, re

df = pd.read_csv(r'E:\Programming\python\NLP\DecoLexO\DecoLexO\example\DECO-Ver5.2-NS-2019-Kernel-DevTest.csv',encoding = 'utf-8-sig')

#첫 행 살리기
col_rgx = re.compile(r'[A-Za-z]{6,}[:\.]?')
first = list(df.columns)
df.loc[0] = first
for val in first:
    m = col_rgx.match(val)
    if m:
        x = first.index(val)
        first[x] = np.nan
    else:
        continue
df.loc[0] = first

##행 이름 변경하기
df = df.fillna('')
sem_rgx = re.compile(r'[Q][A-Z]{3}')  #semantic tagset
syn_rgx = re.compile(r'[Y][A-Z]{3}')  #syntactic tagset
dom_rgx = re.compile(r'[X]{1}[ABCDEFGHIJKLMNOPQRSTUVWYZ]{3}')  #domain tagset
ent_rgx = re.compile(r'[X]{2}[A-Z]{2}') #entity tagset
mor_rgx = re.compile(r'[A-Z]{3}') #morph tagset
seminfo = []
syninfo = []
dominfo = []
entinfo = []
morinfo = []

#컬럼의 총 개수를 l에 저장한다.
#컬럼의 개수 만큼 lemma와 category뒤에 lemma와 category개수인 2를 뺀만큼
#''를 추가해 주어 해당 컬럼 개수 만큼의 리스트 col_nme을 만들어 준다.
l = len(df.columns)
col_nme = ['Lemma','Category']
for i in range(l-2):
    col_nme.append('')
#sem =>SemInfo 뒤에 붙을 숫자 
#syn =>SynInfo 뒤에 붙을 숫자 
#dom =>DomInfo 뒤에 붙을 숫자 
#ent =>EntInfo 뒤에 붙을 숫자 
#mor =>MorInfo 뒤에 붙을 숫자 
sem = 1
syn = 1
dom = 1
ent = 1
mor = 1

#x를 컬럼의 개수 만큼의 숫자로 지정해 준다.
#col_val은 해당 df의 열을 리스트화 시켜준 것이다.
for x in range(0, l):
    col_val = df.iloc[:, x].tolist()
    #cnt가 0이면 일치하는 값을 못 찾았다는 의미로 해석(ex 모두 빈칸인 열을 만났을 때)
    #밑에서 cnt == 0 일때 앞에 정보를 보고 빈칸의 정보를 수정할 때 사용한다.
    cnt = 0
    #k로 col_val의 리스트 요소들을 하나씩 지정해주면서
    #k가 sem_rgx, syn_rgx, dom_rgx, ent_rgx, mor_rgx에 해당되면
    #컬럼에 일치하는 값이 있었다는 의미로 cnt를 1 증가시켜 주고
    #Info뒤에 붙을 숫자를 1씩 증가시켜 주고
    #비효율적인 탐색을 막기 위해 바로 break시켜준다.
    for k in col_val:
        if sem_rgx.match(k):
            col_nme[x] = 'Sem Info' + str(sem)
            sem += 1
            cnt += 1
            break

        elif syn_rgx.match(k):
            col_nme[x] = 'Syn Info' + str(syn)
            syn += 1
            cnt += 1
            break

        elif dom_rgx.match(k):
            col_nme[x] = 'Dom Info' + str(dom)
            dom += 1
            cnt += 1
            break

        elif ent_rgx.match(k):
            col_nme[x] = 'Ent Info' + str(ent)
            ent += 1
            cnt += 1
            break

        elif mor_rgx.match(k):
            col_nme[x] = 'Mor Info' + str(mor)
            mor += 1 
            cnt += 1
            break

    #만약 위에서 일치하는 값을 못찾았을 때(ex 모두 빈칸인 열이었을 때)
    #cnt는 0이므로 앞에 col_nme의 정보를 보고 
    #해당 정보와 일치하는 정보의 Info숫자를 증가시켜준 값을 해당 리스트 위치에 저장해준다.
    if cnt == 0:
        if 'Sem' in col_nme[x-1]:
            col_nme[x] = 'Sem Info' + str(sem)
            sem+=1
            
        elif 'Syn' in col_nme[x-1]:
            col_nme[x] =  'Syn Info' + str(syn)
            syn+=1
        
        elif 'Dom' in col_nme[x-1]:
            col_nme[x] =  'Dom Info' + str(dom)
            dom+=1
            
        elif 'Ent' in col_nme[x-1]:
            col_nme[x] =  'Ent Info' + str(ent)
            ent+=1
            
        elif 'Mor' in col_nme[x-1]:
            col_nme[x] =  'Mor Info' + str(mor)
            mor+=1

df.columns = col_nme

#편집을 원하는 행을 리스트화 하는 함수
#df에는 불러온 데이터프레임을 넣고 col에는 수정 원하는 행 이름을 삽입
def df2first(df,col):
    col_first = list(df.columns)
    x = col_first.index(col)
    res = df.iloc[:,x:x+1].values.tolist()
    res = sum(res,[])
    return res

#add하기    
def add(df, col, add_place, add_text):
    res = []
    first = df2first(df,col)
    if add_place == 'beginning':
        for x in first:
            res.append(add_text + x)
    if add_place == 'ending':
        for x in first:
            res.append(x + add_text)   
    #열 추가
    del df[col]
    df[col] = res
    
    #열 위치 재정렬
    df = df[col_nme]
    
    print(df)

print(df)

Lemma Category Mor Info1 Mor Info2 Mor Info3 Mor Info4 Mor Info5  \
0       가게     NS01       ZNZ       LEO       SLB       HAL       MCO   
1       가결     NS03       ZNZ       LEO       SLB       PHV             
2       가계     NS01       ZNZ       LEO       SLB       NAB             
3       가곡     NS02       ZNX       PXN       NZX       SLB       LEO   
4       가공     NS02       ZNM       LEO       SLB       PSN             
...    ...      ...       ...       ...       ...       ...       ...   
1993    동사     NS01       ZNZ       LEO       SLB       NAB             
1994    동산     NS02       ZNX       PXN       NZX       SLB       LEO   
1995    동상     NS02       ZNX       SXN       NZX       SLB       LEO   
1996    동생     NS02       ZNZ       LEO       SLB       MCO             
1997    동선     NS02       ZNZ       LEO       SLB       PHV             

     Mor Info6 Mor Info7 Mor Info8  ... Dom Info4 Dom Info5 Dom Info6  \
0                                   ...                

In [3]:
len(df)

1998

In [2]:
#입력 받은 단어로 끝나는 단어를 찾아내는 함수
def Ends_With(df, col, word):
    if '+' in word:
            word_list = word.split ('+')
    else:
        word_list = []
        word_list.append (word)
    
    #filtered_list에는 찾고자하는 요소에 매칭되는 단어들을 저장할 것이고
    #filtered_index에는 원본 데이터에 있는 찾은 단어의 인덱스를 저장할 것이다.
    filtered_list = []

    #i는 word_list에 저장된 단어의 개수를 길이로 설정하고
    #j는 원본데이터의 행의 개수를 길이로 설정한다.
    #df는 원본데이터, col은 찾고자 하는 정보(Lemma, Entry, Catgory..), word는 찾고자 하는 단어.
    #df.loc[j, col] => 열마다 돌아가면서 해당 정보와 찾고자 하는 단어로 끝나면
    #answer에 일치 정보의 열을 통째로 저장해준다.
    #이때는 시리즈 형태로 저장되어 있기 때문에 filtered_list에 answer을 list화 한 정보를 넣어주면
    #컬럼 정보는 제외된 순수 데이터 정보만 filteres_list에 저장이된다.
    #filtered_index에는 gui상에서 원본 인덱스보다 +1 된 값으로 나오기 때문에 
    #찾은 데이터의 인덱스 값을 +1해서 넣어준다.
    for i in range (len (word_list)):
        for j in range (len (df)):
            if df.loc[j, col].endswith (word_list[i]):
                result = df.iloc[j]
                filtered_list.append(list(result))

    # result는 series로 저장되어 있기 때문에 to_frame()으로 df화 해주고
    # 현재 row와 column이 바뀌어 저장되어 있기 때문에
    # .T로 transepose() 기능을 주어 row와 column을 바꾸어 준다.
    filtered_header = handle_df.columns.tolist()
    filtered_df = pd.DataFrame(filtered_list, columns=filtered_header)
    filtered_df.head()

    return filtered_df

In [6]:
original_index = []
original_df_index = []
for i in range(len(df)):
    original_df_index.append(df.loc[i,col])


Lemma Category Mor Info1 Mor Info2 Mor Info3 Mor Info4 Mor Info5  \
count   1998     1998      1998      1998      1998      1998      1998   
unique  1998        4         8         4         7        24        18   
top       가든     NS02       ZNZ       LEO       SLB       MCO             
freq       1     1055      1462      1718      1319       543      1440   

       Mor Info6 Mor Info7 Mor Info8  ... Dom Info4 Dom Info5 Dom Info6  \
count       1998      1998      1998  ...      1998      1998      1998   
unique        15         1         1  ...         1         1         1   
top                                   ...                                 
freq        1817      1998      1998  ...      1998      1998      1998   

       Dom Info7 Dom Info8 Dom Info9 Dom Info10 Dom Info11 Dom Info12  \
count       1998      1998      1998       1998       1998       1998   
unique         1        21        23         21         22         19   
top                                 

In [13]:
#입력받은 단어로 시작하는 단어를 찾아내는 함수
def Starts_With(df, col, word):
    # global original_df

    original_index = []
    original_df_index = []
    for i in range(len(df)):
        original_df_index.append(df.loc[i,col])

    if '+' in word:
            word_list = word.split ('+')
    else:
        word_list = []
        word_list.append (word)
    
    #filtered_list에는 찾고자하는 요소에 매칭되는 단어들을 저장할 것이고
    #filtered_index에는 원본 데이터에 있는 찾은 단어의 인덱스를 저장할 것이다.
    filtered_list = []

    #i는 word_list에 저장된 단어의 개수를 길이로 설정하고
    #j는 원본데이터의 행의 개수를 길이로 설정한다.
    #df는 원본데이터, col은 찾고자 하는 정보(Lemma, Entry, Catgory..), word는 찾고자 하는 단어.
    #df.loc[j, col] => 열마다 돌아가면서 해당 정보와 찾고자 하는 단어로 시작하면
    #answer에 일치 정보의 열을 통째로 저장해준다.
    #이때는 시리즈 형태로 저장되어 있기 때문에 filtered_list에 answer을 list화 한 정보를 넣어주면
    #컬럼 정보는 제외된 순수 데이터 정보만 filteres_list에 저장이된다.
    #filtered_index에는 gui상에서 원본 인덱스보다 +1 된 값으로 나오기 때문에 
    #찾은 데이터의 인덱스 값을 +1해서 넣어준다.
    for i in range (len (word_list)):
        for j in range (len (df)):
            if df.loc[j, col].startswith (word_list[i]):
                result = df.iloc[j]
                filtered_list.append(list(result))
                original_index.append(original_df_index.index(df.loc[j, col]))
            

    # result는 series로 저장되어 있기 때문에 to_frame()으로 df화 해주고
    # 현재 row와 column이 바뀌어 저장되어 있기 때문에
    # .T로 transepose() 기능을 주어 row와 column을 바꾸어 준다.
    filtered_header = df.columns.tolist()
    filtered_df = pd.DataFrame(filtered_list, columns=filtered_header)
    filtered_df.head()

    print(original_index)

    return filtered_df

In [14]:
print(Starts_With(df, 'Lemma', '가'))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
   Lemma Category Mor Info1 Mor Info2 Mor Info3 Mor Info4 Mor Info5 Mor Info6  \
0     가게     NS01       ZNZ       LEO       SLB       HAL       MCO             
1     가결     NS03       ZNZ       LEO       SLB       PHV                       
2     가계     NS01       ZNZ       LEO       SLB       NAB                       
3     가곡     NS02       ZNX       PXN       NZX       SLB       LEO       HAL   
4     가공     NS02       ZNM       LEO       SLB       PSN                       
..   ...      ...       ...       ...       ...       ...       ...       ...   
76    가택     NS02       ZNZ       LEO       SLB       MCO                       
77   가톨릭     NS02       ZNF       SIM

In [1]:
original_index = [3, 453, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 960, 1006]


        for x in original_index:
            for y in range(0, len(handl                filtered_df.iloc[x, :] = handle_df.iloc[y, :]


NameError: name 'df' is not defined